## Database Design and SQL queries

The notebook manages a retail database using PostgreSQL. It includes functions to establish database connections, create necessary tables (customers, addresses, orders, order lines), and insert initial data. It also manages data integrity through foreign key constraints and includes utilities for updating and deleting records. This foundational tool is designed for robust database management, facilitating data entry, updates, and ensuring relational links between different data entities in a retail business setting.

In [23]:
from posgresql_util import pgdao

In [24]:
from posgresql_util import psycopg2

In [25]:
def create_connection():
    """Establishes and returns a PostgreSQL database connection."""
    conn = psycopg2.connect(host="localhost",port=5432,database="your_db_name",user="your_username",password="your_password")
    return conn

def create_tables(cursor):
    """Creates necessary tables in the database."""
    customer_table = """
    CREATE TABLE IF NOT EXISTS customers (
        customer_id SERIAL PRIMARY KEY,
        telephone VARCHAR(10) UNIQUE NOT NULL,
        email VARCHAR(255) UNIQUE NOT NULL,
        first_name VARCHAR(100) NOT NULL,
        last_name VARCHAR(100) NOT NULL
    );
    """
    address = """
    CREATE TABLE IF NOT EXISTS addresses (
        address_id SERIAL PRIMARY KEY,
        customer_id INT NOT NULL,
        street VARCHAR(255) NOT NULL,
        city VARCHAR(100) NOT NULL,
        state VARCHAR(100) NOT NULL,
        zip_code VARCHAR(15) NOT NULL,
        address_type VARCHAR(50) NOT NULL,
        is_active INT NOT NULL
    );
    """
    order = """
    CREATE TABLE IF NOT EXISTS orders (
        order_id SERIAL PRIMARY KEY,
        customer_id INT NOT NULL,
        order_date TIMESTAMP,
        billing_address_id INT NOT NULL,
        order_type VARCHAR(100) NOT NULL
    );
    """
    order_line = """
    CREATE TABLE IF NOT EXISTS orders_lines (
        order_line_id SERIAL PRIMARY KEY,
        product_id INT NOT NULL,
        order_id INT NOT NULL,
        shipment_address_id INT NOT NULL
    );
    """
    cursor.execute(customer_table)
    cursor.execute(address)
    cursor.execute(order)
    cursor.execute(order_line)

def add_constraints(cursor):
    """Adds foreign key constraints to the tables."""
    order_constraint = """
    ALTER TABLE orders
        ADD CONSTRAINT fk_order_customer_id FOREIGN KEY (customer_id) REFERENCES customers (customer_id) ON DELETE CASCADE,
        ADD CONSTRAINT fk_order_billing_address_id FOREIGN KEY (billing_address_id) REFERENCES addresses(address_id) ON DELETE CASCADE
    """
    order_line_constraint = """
    ALTER TABLE orders_lines
        ADD CONSTRAINT fk_order_line_shipment_address_id FOREIGN KEY (shipment_address_id) REFERENCES addresses(address_id) ON DELETE CASCADE,
        ADD CONSTRAINT fk_order_line_order_id FOREIGN KEY (order_id) REFERENCES orders(order_id) ON DELETE CASCADE
    """
    cursor.execute(order_constraint)
    cursor.execute(order_line_constraint)

In [26]:
# Functions to drop constraints and tables
def drop_constraints(cursor):
    cursor.execute("ALTER TABLE orders DROP CONSTRAINT fk_order_customer_id;")
    cursor.execute("ALTER TABLE orders DROP CONSTRAINT fk_order_billing_address_id;")
    cursor.execute("ALTER TABLE orders_lines DROP CONSTRAINT fk_order_line_shipment_address_id;")
    cursor.execute("ALTER TABLE orders_lines DROP CONSTRAINT fk_order_line_order_id;")
def drop_tables(cursor):
    cursor.execute("DROP TABLE IF EXISTS customers")
    cursor.execute("DROP TABLE IF EXISTS orders")
    cursor.execute("DROP TABLE IF EXISTS addresses")
    cursor.execute("DROP TABLE IF EXISTS customer_addresses")
    cursor.execute("DROP TABLE IF EXISTS orders_lines")

In [27]:
# Functions to insert values to table
def insert_customers(cursor):
    """Inserts initial data into the 'customers' table."""
    query = """
    INSERT INTO customers (telephone, email, first_name, last_name) VALUES
    ('8378139380', 'john.doe@google.com', 'John', 'Doe'),
    ('4127837489', 'jane.smith@google.com', 'Jane', 'Smith');
    """
    cursor.execute(query)

def insert_addresses(cursor):
    """Inserts initial data into the 'addresses' table."""
    query = """
    INSERT INTO addresses (customer_id, street, city, state, zip_code, address_type, is_active) VALUES
    (1, '1267 Elf Street', 'Smalltown', 'CA', '12344', 'home', 0),
    (1, '1234 Elm Street', 'Smalltown', 'CA', '12345', 'home', 1),
    (1, '5678 Oak Street', 'Smalltown', 'CA', '12345', 'office', 1),
    (2, '1234 Pine Street', 'Bigcity', 'NY', '67890', 'home', 1),
    (2, '5678 Maple Street', 'Bigcity', 'NY', '67890', 'office', 1);
    """
    cursor.execute(query)

def insert_orders(cursor):
    """Inserts initial data into the 'orders' table."""
    query = """
    INSERT INTO orders (customer_id, order_date, billing_address_id, order_type) VALUES
    (1, '2023-05-01 10:00:00', 1, 'delivery'),
    (2, '2023-05-02 15:30:00', 2, 'in-store'),
    (2, '2023-05-03 15:30:00', 2, 'in-store'),
    (1, '2023-05-04 17:30:00', 2, 'in-store');
    """
    cursor.execute(query)

def insert_orders_lines(cursor):
    """Inserts initial data into the 'orders_lines' table."""
    query = """
    INSERT INTO orders_lines (product_id, order_id, shipment_address_id) VALUES
    (101, 1, 1),
    (102, 1, 2),
    (103, 2, 4),
    (104, 2, 4),
    (104, 3, 5),
    (104, 4, 3);
    """
    cursor.execute(query)

In [28]:
def execute():
    """Main function to manage database operations."""
    conn = None
    cur = None
    try:
        
        conn = create_connection()
        cur = conn.cursor() # creates a new cursor object from your connection
        
        conn_dict = {'host':'localhost' ,'port' :'5432', 'database' :'your_db_name' , 'user':'your_username', 'password':'your_password'}
        #setting the connection
        p1 = pgdao(conn_dict)
        result = p1.query('select * from information_schema.tables')
        
        table = ['customers','addresses','orders_lines','orders']
        result_tables = list(result['TABLE_NAME'])
        # List comprehension to filter the names
        found_tables = [table for table in table if table in result_tables]
        # Check if the list of found tables is not empty
        if found_tables:
            print(f"Found tables: {found_tables}")
            print('Drop tables')
            drop_constraints(cur)
            drop_tables(cur)
        
        print('Creating Tables')
        create_tables(cur)
        conn.commit()  # Commit changes after creating tables
        
        print('Adding Constraints')
        add_constraints(cur)
        conn.commit()  # Commit changes after adding constraints
        
        print('Insert Value into Tables')
        # Insert data into tables
        insert_customers(cur)
        insert_addresses(cur)
        insert_orders(cur)
        insert_orders_lines(cur)
        conn.commit()  # Commit changes after inserting data
                            
    except Exception as e:
        print(f"An error occurred: {e}")
        if conn:
            conn.rollback()  # Rollback in case of error
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()


In [29]:
execute()

Executing Query : 
b'select * from information_schema.tables'
Found tables: ['customers', 'addresses', 'orders_lines', 'orders']
Drop tables
Creating Tables
Adding Constraints
Insert Value into Tables


In [30]:
#Connection Declaration
conn_dict = {'host':'localhost' ,'port' :'5432', 'database' :'your_db_name' , 'user':'your_username', 'password':'your_password'}
#setting the connection
p1 = pgdao(conn_dict)

In [31]:
p1.query("SELECT * FROM orders")


Executing Query : 
b'SELECT * FROM orders'


,ORDER_ID,CUSTOMER_ID,ORDER_DATE,BILLING_ADDRESS_ID,ORDER_TYPE
0,1,1,2023-05-01 10:00:00,1,delivery
1,2,2,2023-05-02 15:30:00,2,in-store
2,3,2,2023-05-03 15:30:00,2,in-store
3,4,1,2023-05-04 17:30:00,2,in-store


In [32]:
p1.query("SELECT * FROM orders_lines")


Executing Query : 
b'SELECT * FROM orders_lines'


,ORDER_LINE_ID,PRODUCT_ID,ORDER_ID,SHIPMENT_ADDRESS_ID
0,1,101,1,1
1,2,102,1,2
2,3,103,2,4
3,4,104,2,4
4,5,104,3,5
5,6,104,4,3


In [33]:
p1.query("SELECT * FROM addresses")


Executing Query : 
b'SELECT * FROM addresses'


,ADDRESS_ID,CUSTOMER_ID,STREET,CITY,STATE,ZIP_CODE,ADDRESS_TYPE,IS_ACTIVE
0,1,1,1267 Elf Street,Smalltown,CA,12344,home,0
1,2,1,1234 Elm Street,Smalltown,CA,12345,home,1
2,3,1,5678 Oak Street,Smalltown,CA,12345,office,1
3,4,2,1234 Pine Street,Bigcity,NY,67890,home,1
4,5,2,5678 Maple Street,Bigcity,NY,67890,office,1


In [34]:
p1.query("SELECT * FROM customers")

Executing Query : 
b'SELECT * FROM customers'


,CUSTOMER_ID,TELEPHONE,EMAIL,FIRST_NAME,LAST_NAME
0,1,8378139380,john.doe@google.com,John,Doe
1,2,4127837489,jane.smith@google.com,Jane,Smith
